In [14]:
import pandas as pd

# Quantifying Monster Actions
In this notebook, actions taken by a monster will be quantified. This will mostly focus on attack actions. Monsters typically look something like the following:

![title](img/Aboleth_actions.png)

The descriptions of each action was scraped and stored in `actions.csv`.

In [13]:
df = pd.read_csv('actions.csv')
df.head(10)

,Monster Name,Name,Description,Type
0,Aboleth,Amphibious.,The aboleth can breathe air and water.,extra
1,Aboleth,Mucous Cloud.,"While underwater, the aboleth is surrounded by...",extra
2,Aboleth,Probing Telepathy.,If a creature communicates telepathically with...,extra
3,Aboleth,Multiattack.,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle.,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail.,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave .,The aboleth targets one creature it can see wi...,action
7,Aboleth,Detect.,The aboleth makes a Wisdom (Perception) check.,legend
8,Aboleth,Tail Swipe.,The aboleth makes one tail attack.,legend
9,Aboleth,Psychic Drain .,One creature charmed by the aboleth takes 10 (...,legend


The part of the monster that has a rigid application in the game mechanics is the "actions" section. This is where each monsters attacks are described. There are also "extras" that tend to contain special information about about what a monster can and can't do (jump high, breathe water, etc.). Some monsters have "Legendary Actions" that can be played at the end of each characters turn, but only once per monster turn. For now, I will focus on just 'actions'. 

In [35]:
df_actions = df[df.Type == 'action']
df_actions.head(10)

,Monster Name,Name,Description,Type
3,Aboleth,Multiattack.,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle.,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail.,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave .,The aboleth targets one creature it can see wi...,action
13,Deva,Multiattack.,The deva makes two melee attacks.,action
14,Deva,Mace.,"Melee Weapon Attack: +8 to hit, reach 5 ft., o...",action
15,Deva,Healing Touch .,The deva touches another creature. The target ...,action
16,Deva,Change Shape.,The deva magically polymorphs into a humanoid ...,action
21,Planetar,Multiattack.,The planetar makes two melee attacks.,action
22,Planetar,Greatsword.,"Melee Weapon Attack: +12 to hit, reach 5 ft., ...",action


I don't like the '.' in the name of each attack, and I do not trust that the spacing is consistant. Let's strip it. Let's also strip any potential hidden white space. Then, let's look at all of the unique action names that there are.

In [36]:
df_actions.Name = df_actions.Name.apply(lambda x : x.strip('. '))

In [37]:
df_actions.head()

,Monster Name,Name,Description,Type
3,Aboleth,Multiattack,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave,The aboleth targets one creature it can see wi...,action
13,Deva,Multiattack,The deva makes two melee attacks.,action


In [58]:
df_actions.groupby("Name").Type.count().head(20)

Name
Acid Breath           7
Acid Spray            1
Animate Chains        1
Animate Trees         1
Antennae              1
Battleaxe             1
Beak                 12
Beaks                 1
Beard                 1
Bite                138
Bites                 6
Blinding Breath       1
Blinding Spittle      1
Blood Drain           1
Breath Weapons       20
Chain                 1
Change Shape         11
Charm                 2
Claw                 54
Claws                29
Name: Type, dtype: int64

It looks like there are some plural versions of actions. This is worth investigating. Let's take a look at the difference between 'Claw' and 'Claws'.

In [83]:
df_claw = df_actions[df_actions.Name == 'Claw']
print(df_claw.iloc[1].Description)
df_claw.head()

Melee Weapon Attack: +10 to hit, reach 10 ft., one target. Hit: 15 (3d6 + 5) slashing damage.


,Monster Name,Name,Description,Type
122,Hezrou,Claw,"Melee Weapon Attack: +7 to hit, reach 5 ft., o...",action
133,Nalfeshnee,Claw,"Melee Weapon Attack: +10 to hit, reach 10 ft.,...",action
150,Barbed Devil,Claw,"Melee Weapon Attack: +6 to hit, reach 5 ft., o...",action
162,Bone Devil,Claw,"Melee Weapon Attack: +8 to hit, reach 10 ft., ...",action
200,Pit Fiend,Claw,"Melee Weapon Attack: +14 to hit, reach 10 ft.,...",action


In [84]:
df_claws = df_actions[df_actions.Name == 'Claws']
print(df_claws.iloc[1].Description)
df_claws.head()

Melee Weapon Attack: +2 to hit, reach 5 ft., one target. Hit: 5 (2d4) slashing damage.


,Monster Name,Name,Description,Type
75,Chimera,Claws,"Melee Weapon Attack: +7 to hit, reach 5 ft., o...",action
111,Dretch,Claws,"Melee Weapon Attack: +2 to hit, reach 5 ft., o...",action
138,Quasit,Claws,"Melee Weapon Attack: +4 to hit, reach 5 ft., o...",action
184,Ice Devil,Claws,"Melee Weapon Attack: +10 to hit, reach 5 ft., ...",action
595,Ettercap,Claws,"Melee Weapon Attack: +4 to hit, reach 5 ft., o...",action
